building the models based on the saved weights:

In [1]:

import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),

    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.4),

    nn.Linear(128, 2)
).to(device)

checkpoint = torch.load("best_deepfake_detector.pt", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
  (15): MaxPool2d(kernel_size=2, stri

In [3]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])


### Folder inference function:

In [4]:
def evaluate_folder(folder_path, true_label):
    """true_label: 0=real, 1=fake"""
    files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    correct = 0
    total = 0
    errors = []

    for filename in tqdm(files, desc=f"Evaluation {os.path.basename(folder_path)}"):
        image_path = os.path.join(folder_path, filename)
        try:
            image = Image.open(image_path).convert("RGB")
            image = transform(image).unsqueeze(0).to(device)

            with torch.no_grad():
                outputs = model(image)
                probs = torch.softmax(outputs, dim=1)
                pred = torch.argmax(probs, dim=1).item()

            if pred == true_label:
                correct += 1
            else:
                errors.append((filename, probs.cpu().numpy()))
            total += 1

        except Exception as e:
            print(f"Error in {filename}: {e}")

    accuracy = correct / total * 100 if total > 0 else 0
    return correct, total, accuracy, errors


#### Launch:

In [5]:

print("=" * 50)

correct_fake, total_fake, acc_fake, _ = evaluate_folder("data/val/fake", true_label=0)
print(f"\n📁 FAKE  → {correct_fake}/{total_fake} corrects  |  Accuracy: {acc_fake:.2f}%")

correct_real, total_real, acc_real, _ = evaluate_folder("data/val/real", true_label=1)
print(f"📁 REAL  → {correct_real}/{total_real} corrects  |  Accuracy: {acc_real:.2f}%")


total_correct = correct_fake + correct_real
total_all = total_fake + total_real
global_acc = total_correct / total_all * 100 if total_all > 0 else 0

print("\n" + "=" * 50)
print(f"✅ global accuracy : {total_correct}/{total_all}  →  {global_acc:.2f}%")
print("=" * 50)

Evaluation fake: 100%|██████████| 8004/8004 [00:16<00:00, 474.41it/s]



📁 FAKE  → 7303/8004 corrects  |  Accuracy: 91.24%


Evaluation real: 100%|██████████| 9763/9763 [00:13<00:00, 728.51it/s]

📁 REAL  → 9011/9763 corrects  |  Accuracy: 92.30%

✅ global accuracy : 16314/17767  →  91.82%


---

### Single image inference:

In [6]:
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()

    return predicted_class, probabilities.cpu().numpy()

#### Launch:

In [8]:
# -------------------------
# TEST
# -------------------------
image_path = "data/val/real/00009.jpg"
#image_path = "data/val/fake/fake_000153.jpg"
pred_class, probs = predict_image(image_path)

print("Predicted class :", pred_class)
print("Probas :", probs)

Predicted class : 1
Probas : [[0.0068441 0.9931559]]
